In [126]:
import cv2
import numpy as np

1. 将图像只保留红色通道。

In [127]:
# 读取图像
image = cv2.imread('D:\\1.jpg')

# 分离通道
b, g, r = cv2.split(image)

# 只保留红色通道，将绿色和蓝色通道设置为0
zeros = np.zeros_like(r)
red_only = cv2.merge([zeros, zeros, r])

# 转换为灰度图像
gray = cv2.cvtColor(red_only, cv2.COLOR_BGR2GRAY)


2. 将灰度值足够大的提取出来，并去除噪点（当前方法的前提是激光点远大于噪点）。

In [128]:
# 应用阈值
_, binary = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)

# 定义腐蚀核的大小和形状
kernel = np.ones((10, 10), np.uint8)  # 可以根据需要调整核的大小和形状
# 进行腐蚀操作
binary_eroded = cv2.erode(binary, kernel, iterations=2)

image1 = cv2.resize(gray,(500,500))
image2 = cv2.resize(binary_eroded,(500,500))
horizontal_concat = cv2.hconcat([image1, image2])
cv2.imshow('binary', horizontal_concat)
cv2.waitKey(0)
cv2.destroyAllWindows()

3. 根据轮廓面积和形状判断。

In [132]:
def is_circle_like(contour, threshold=0.7):
    area = cv2.contourArea(contour)
    ((x, y), radius) = cv2.minEnclosingCircle(contour)
    circle_area = np.pi * (radius ** 2)
    
    # 比较轮廓面积和外接圆面积
    print("1",area / circle_area)
    return area / circle_area > threshold

# 查找轮廓
contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# cv2.drawContours(image, contours, -1, (255, 255, 255), 2)

# 绘制轮廓
for cnt in contours:
    # 计算轮廓的面积
    area = cv2.contourArea(cnt)
    
    # 过滤掉小面积的轮廓，并检查是否接近圆形
    if 50 < area and is_circle_like(cnt):
        # 绘制轮廓
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, 'G', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 10)

# 显示结果图像
cv2.imshow('Detected Laser', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


1 0.1315865831607053
1 0.7109611129652225
1 0.18435948603951705
1 0.3361954911131761
